In [16]:
import pandas as pd
import numpy as np
from datetime import datetime, timedelta

import openpyxl as op

In [17]:
# Paramétrage :

# from final_tab.ipynb
# Define the angle range and step
start_angle = -60
end_angle = 60
step_angle = 61

# Create the array of angles
angles = np.arange(start_angle, end_angle + 1, step_angle, dtype=object)
titres_colonnes = np.append(angles, np.array("temoin", dtype=object), axis=None)

# Paramètres annuels
###### # from final_tab.ipynb
start_date = datetime(year = 2023, month = 1, day = 1, hour = 0)
end_date = datetime(year = 2024, month = 1, day = 1, hour = 0)
print((end_date-start_date).days)

# Paramètres horaires
###### # from final_tab.ipynb
HEURE_DEB = 6
HEURE_FIN = 22
HEURE_BLANCHES = 24 - (HEURE_FIN-HEURE_DEB)
NB_HEURES_ANNEES_ETUD = (HEURE_FIN-HEURE_DEB)*(end_date-start_date).days
TOT_HOYS = (end_date-start_date).days*24

365


In [18]:
titres_colonnes

array([-60, 1, 'temoin'], dtype=object)

In [19]:
# Lecture du tableau généré aux étapes précédentes
df = pd.read_excel("results/tabmean_5840_60-1_temoin.xlsx",sheet_name="Sheet1",header=None)
df.columns = titres_colonnes
print(df)

           -60         1    temoin
0     0.000000  0.000000  0.000000
1     0.049168  0.049168  0.049168
2     0.181327  0.181327  0.181327
3     0.312087  0.312087  0.312087
4     0.343043  0.343043  0.343043
...        ...       ...       ...
5835  0.001077  0.001077  0.001077
5836  0.000000  0.000000  0.000000
5837  0.000000  0.000000  0.000000
5838  0.000000  0.000000  0.000000
5839  0.000000  0.000000  0.000000

[5840 rows x 3 columns]


In [20]:
# Étape 1 : Ajouter autant de lignes nulles qu'HEURE_DEB si positif
if HEURE_DEB > 0:
    null_rows_start = pd.DataFrame(0, index=range(HEURE_DEB), columns=titres_colonnes)
    df2 = pd.concat([null_rows_start, df], ignore_index=True)

# Étape 2 : Ajouter 8 lignes vides toutes les 16 lignes jusqu'à atteindre 8760 lignes
if HEURE_BLANCHES > 0:
    empty_rows = pd.DataFrame(0, index=range(HEURE_BLANCHES), columns=titres_colonnes)

    for idx in range(HEURE_FIN, TOT_HOYS, 24):
        df2 = pd.concat([df2.iloc[:idx], empty_rows, df2.iloc[idx:]], ignore_index=True)

# Étape 3 : Enlever les lignes en trop
if df2.shape[0] > TOT_HOYS :
    surplus = df2.shape[0] - TOT_HOYS
    df3 = df2.iloc[:-surplus]

In [21]:
print(df3[0:24])

         -60         1    temoin
0   0.000000  0.000000  0.000000
1   0.000000  0.000000  0.000000
2   0.000000  0.000000  0.000000
3   0.000000  0.000000  0.000000
4   0.000000  0.000000  0.000000
5   0.000000  0.000000  0.000000
6   0.000000  0.000000  0.000000
7   0.049168  0.049168  0.049168
8   0.181327  0.181327  0.181327
9   0.312087  0.312087  0.312087
10  0.343043  0.343043  0.343043
11  0.305425  0.305425  0.305425
12  0.320081  0.320081  0.320081
13  0.376153  0.376153  0.376153
14  0.334291  0.334291  0.334291
15  0.178140  0.178140  0.178140
16  0.043632  0.043632  0.043632
17  0.001077  0.001077  0.001077
18  0.000000  0.000000  0.000000
19  0.000000  0.000000  0.000000
20  0.000000  0.000000  0.000000
21  0.000000  0.000000  0.000000
22  0.000000  0.000000  0.000000
23  0.000000  0.000000  0.000000


In [22]:
# Créer un DataFrame avec les heures de l'année (de 0 à 8759) en tant qu'index
hours_in_year = range(8760)
dfX = pd.DataFrame(index=hours_in_year)

# Ajouter la colonne 'month_day_hour' sans déclencher l'avertissement
df3['month_day_hour'] = pd.to_datetime(dfX.index.map(lambda x: start_date + timedelta(hours=x)))

# Réorganiser les colonnes dans df3
cols = ['month_day_hour'] + [col for col in df3 if col != 'month_day_hour']
df3 = df3[cols]
print(df3.head())

       month_day_hour  -60    1  temoin
0 2023-01-01 00:00:00  0.0  0.0     0.0
1 2023-01-01 01:00:00  0.0  0.0     0.0
2 2023-01-01 02:00:00  0.0  0.0     0.0
3 2023-01-01 03:00:00  0.0  0.0     0.0
4 2023-01-01 04:00:00  0.0  0.0     0.0


c:\Users\maceo.valente\.virtualenvs\TSAgriPV-XqNYGekU\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


Import scenario :
1 = True = Backtracking (productible max)
0 = False = Effacement (irradiance au sol maximum)

In [23]:
from scenario import Scenario

## TESTS ##
# Instancier des objets scénarios
s1 = None
xl_path = "scenarios/sc_test1.xlsx"
s1 = Scenario(file_path=xl_path)
# print(s1.scenario[:,:])

s2 = Scenario(file_path=xl_path)
print(s2.scenario[:,:])



[[ True  True  True  True  True  True  True  True  True  True  True  True
   True  True  True  True  True  True  True  True  True  True  True  True]
 [ True  True  True  True  True  True  True  True  True  True  True  True
   True  True  True  True  True  True  True  True  True  True  True  True]
 [ True  True  True  True  True  True  True  True  True  True  True  True
   True  True  True  True  True  True  True  True  True  True  True  True]
 [ True  True  True  True  True  True  True  True  True  True  True  True
   True  True  True  True  True  True  True  True  True  True  True  True]
 [ True  True  True  True  True  True  True  True  True  True  True False
  False False False False False False  True  True  True  True  True  True]
 [ True  True  True  True  True  True  True  True  True  True  True False
  False False False False False False  True  True  True  True  True  True]
 [ True  True  True  True  True  True  True  True  True  True  True False
  False  True  True  True False 

In [24]:
s2.scenario.shape

(12, 24)

Création équivalent 1/0

In [25]:
# Création_tab 1
sc10 = np.full((12,24), 0, dtype = "int8")

for i in range(s2.scenario.shape[0]):  # Pour chaque ligne
    for j in range(s2.scenario.shape[1]):  # Pour chaque colonne
        if s2.scenario[i, j] == True:
            sc10[i, j] = 1

print(sc10)

[[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 0 0 0 0 0 0 0 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 0 0 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 0 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]
 [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]]


Création tab_1

In [26]:
# Nombre de lignes souhaité
n_rows = 8760

# Création d'un DataFrame avec des valeurs par défaut (par exemple, 0)
tab_1 = pd.DataFrame({
    'hoys': df3['month_day_hour'],
    'effacement': np.ones(n_rows),
    'angle': np.zeros(n_rows),
    'irradiance_sol': np.zeros(n_rows)
})


Import PVSyst

In [27]:
# Chemin vers le fichier Excel
file_path = 'PVSystTestFile.xlsm'

# Nom de l'onglet à lire
sheet_names = ['Tests','Illim']

# Lire l'onglet spécifique
pvsyst = pd.read_excel(file_path, sheet_name=sheet_names)

In [28]:
pvsyst_illim = pvsyst['Illim'].drop(index=list(range(12))).drop(columns=['Unnamed: 4', 'Unnamed: 5']).reset_index(drop=True)
pvsyst_illim.columns = ['Date','E_Grid','PR','PhiAng']
print(pvsyst_illim)

                     Date E_Grid PR PhiAng
0     1990-01-01 00:00:00 -3.048  0      0
1     1990-01-01 01:00:00 -3.048  0      0
2     1990-01-01 02:00:00 -3.048  0      0
3     1990-01-01 03:00:00 -3.048  0      0
4     1990-01-01 04:00:00 -3.048  0      0
...                   ...    ... ..    ...
8755  1990-12-31 19:00:00 -3.048  0      0
8756  1990-12-31 20:00:00 -3.048  0      0
8757  1990-12-31 21:00:00 -3.048  0      0
8758  1990-12-31 22:00:00 -3.048  0      0
8759  1990-12-31 23:00:00 -3.048  0      0

[8760 rows x 4 columns]


In [29]:
pvsyst_tests = pvsyst['Tests'].drop(index=list(range(12))).drop(columns=pvsyst['Tests'].columns[4:]).reset_index(drop=True)
pvsyst_tests.columns = ['Date', 'PhiAng', 'E_Grid_SunTrack', 'Egrid Effectif']
print(pvsyst_tests)

                     Date PhiAng E_Grid_SunTrack Egrid Effectif
0     1990-01-01 00:00:00      0          -3.048              1
1     1990-01-01 01:00:00      0          -3.048              1
2     1990-01-01 02:00:00      0          -3.048              1
3     1990-01-01 03:00:00      0          -3.048              1
4     1990-01-01 04:00:00      0          -3.048              1
...                   ...    ...             ...            ...
8755  1990-12-31 19:00:00      0          -3.048             12
8756  1990-12-31 20:00:00      0          -3.048             12
8757  1990-12-31 21:00:00      0          -3.048             12
8758  1990-12-31 22:00:00      0          -3.048             12
8759  1990-12-31 23:00:00      0          -3.048             12

[8760 rows x 4 columns]


Remplir le tab_1 :

In [30]:
# Remplissage de la colonne 'effacement'
# Parcourir scénario, pour chaque case 0 (False), changer les 1 (panneaux en suntracking) de la colonne 'effacement' en 0 (panneaux effacés)

for i in range(s1.scenario.shape[0]) :  # Pour chaque ligne
    for j in range(s1.scenario.shape[1]) :  # Pour chaque colonne
        if s1.scenario[i,j] == False :
            tab_1.loc[(tab_1['hoys'].dt.hour == j) & (tab_1['hoys'].dt.month == i+1), 'effacement'] = 0

In [31]:
# Remplissage de la colonne 'angle'
# Parcourir tableau_1, en fonction de 'effacement', remplir 'angle' et 'irradiance'
for i in range(tab_1.shape[0]) :
    if tab_1.at[i,'effacement'] == 0:
        #aller chercher irradiance au sol maximum dans tab_final (df3 dans ce .ipynb), et retrouver l'angle correspondant (nom de colonne)
        tab_1.loc[i,'irradiance_sol'] = df3.drop(columns='month_day_hour').iloc[i].max() # Irradiance max
        tab_1.loc[i,'angles'] = df3.drop(columns='month_day_hour').iloc[i].idxmax() # Colonne dont l'irradiance est max
        # /!\ Si égalité entre plusieurs angles, va choisir l'angle le moins élevé (-60 à priori)
    else: #'effacement' == 1
        # pvsyst_illim.loc[...] = angle PVSYST optimal, arrondir, puis trouver irradiance au sol correspondante
        tab_1.loc[i,'irradiance_sol'] = df3.at[i,round(pvsyst_illim.loc[i,'PhiAng']/step_angle)*step_angle]
        tab_1.loc[i,'angles'] = round(pvsyst_illim.loc[i,'PhiAng']/step_angle)*step_angle

# Tableau_1 rempli, il ne reste qu'à calculer les rapports entre zone témoin /!\(à intégrer)/!\ et zone d'étude
  

KeyError: 0

In [ ]:
# print(tab_1[(tab_1['effacement'] < 1) & (tab_1['hoys'].dt.month == 9)])
print(tab_1)

                    hoys  effacement  angle  irradiance_sol  angles
0    2023-01-01 00:00:00         1.0    0.0             0.0     0.0
1    2023-01-01 01:00:00         1.0    0.0             0.0     0.0
2    2023-01-01 02:00:00         1.0    0.0             0.0     0.0
3    2023-01-01 03:00:00         1.0    0.0             0.0     0.0
4    2023-01-01 04:00:00         1.0    0.0             0.0     0.0
...                  ...         ...    ...             ...     ...
8755 2023-12-31 19:00:00         1.0    0.0             0.0     0.0
8756 2023-12-31 20:00:00         1.0    0.0             0.0     0.0
8757 2023-12-31 21:00:00         1.0    0.0             0.0     0.0
8758 2023-12-31 22:00:00         1.0    0.0             0.0     0.0
8759 2023-12-31 23:00:00         1.0    0.0             0.0     0.0

[8760 rows x 5 columns]


Calcul liste de taille 365 concernant rapport de irradiance_zone_témoin sur irradiance_zone_étude

In [ ]:
df_daily = tab_1.groupby(tab_1['hoys'].dt.date)['irradiance_sol'].sum().reset_index()
df_daily.columns = ['date', 'irradiance_sol_day']

print(df_daily)

           date  irradiance_sol_day
0    2023-01-01            2.444301
1    2023-01-02            0.339211
2    2023-01-03            3.506907
3    2023-01-04            2.824320
4    2023-01-05            0.552571
..          ...                 ...
360  2023-12-27            1.630395
361  2023-12-28            0.160914
362  2023-12-29            1.213723
363  2023-12-30            3.113947
364  2023-12-31            3.771072

[365 rows x 2 columns]


###########
###########
#### TESTS
###########
###########

In [33]:
df3[5:25]

,month_day_hour,-60,1,temoin
5,2023-01-01 05:00:00,0.000000,0.000000,0.000000
6,2023-01-01 06:00:00,0.000000,0.000000,0.000000
7,2023-01-01 07:00:00,0.049168,0.049168,0.049168
8,2023-01-01 08:00:00,0.181327,0.181327,0.181327
9,2023-01-01 09:00:00,0.312087,0.312087,0.312087
10,2023-01-01 10:00:00,0.343043,0.343043,0.343043
11,2023-01-01 11:00:00,0.305425,0.305425,0.305425
12,2023-01-01 12:00:00,0.320081,0.320081,0.320081
13,2023-01-01 13:00:00,0.376153,0.376153,0.376153
14,2023-01-01 14:00:00,0.334291,0.334291,0.334291
